# 準備

In [ ]:
!pip install --upgrade xlrd==1.2.0

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import urllib.parse
import urllib.request
import json
import os
import pandas as pd
import numpy as np
import xlrd
import openpyxl

In [ ]:
!pip install openpyxl

# 企業活動のCO2

## 単体チェック

EXCELをインポートして整形する関数

In [ ]:
def get_co2(path):

  # Bookをインポートしてシート名をリストに格納
  book = pd.ExcelFile(path)
  sheets = book.sheet_names

  # ファイル名からIDと事業者名を取得
  filename = path.split('/')[6]
  id = filename.split('_')[0]
  company = filename.split('_')[1].replace('.xlsx', '')
  
  # CO2排出量のシートをDataFrameに格納
  df = book.parse(sheets[4])
  
  df = df[['Unnamed: 4','Unnamed: 9','Unnamed: 15','Unnamed: 17','Unnamed: 19']]
  df = df.rename(columns={'Unnamed: 4' : '種類','Unnamed: 9' : '使用量','Unnamed: 15' : '単位','Unnamed: 17' : '排出係数','Unnamed: 19' : 'CO2排出量'})
  df = df.dropna(subset=['種類','使用量','CO2排出量']).reset_index(drop=True)
  
  # 余計なデータを削除
  df = df[df['CO2排出量'] != '－']

  df = df.assign(年次 = '',会社名 = company,ID = id)
  
  for index, row in df.iterrows():
    
    if type(row['CO2排出量']) is str:
      time = '2021年' if index == 0 else '2013年'
      df = df.drop(index)
  
    df['年次'][index] = time

  # 荷役機械のデータが紛れ込むので削除
  df = df[df['排出係数'] != 'LPG']
  df = df[df['排出係数'] != 'ケミカル']
  df = df[df['排出係数'] != '石炭']
  df = df[df['排出係数'] != 'バン頁']
  df = df[df['排出係数'] != '珪砂']
  
  # 改行を削除
  df = df.replace( '\n', '', regex=True)

  # CO2排出量を整数に変換
  df['CO2排出量'] = df['CO2排出量'].astype('int')

  # 排出区分を追加する
  df_type = pd.read_csv('/content/drive/MyDrive/INBOX/企業活動排出区分.csv', encoding = "shift-jis")
  df_type = df_type[~df_type.duplicated()]
 
  df = pd.merge(df, df_type, on=['種類'] , how='left')
  df = df.fillna({'区分': '電気'})

  return df


パスを指定してテスト

In [ ]:
path = '/content/drive/MyDrive/INBOX/テスト/専39_日本製鉄.xlsx'
df = get_co2(path)
df

/usr/local/lib/python3.7/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,使用量,単位,排出係数,CO2排出量,年次,会社名,ID,算出方法,種類,区分


合計値を確認しておく

In [ ]:
# 合計値の確認
df_sum = df.groupby(['年次']).agg({'CO2排出量': 'sum'}).reset_index()
df_sum

,年次,CO2排出量
0,2013年,106541639
1,2021年,107106085


## CO2排出明細の作成

ディレクトリ内のファイル一覧を取得する

In [ ]:
def get_dataframe():

  files = os.listdir('drive/MyDrive/INBOX/テスト/')
  
  df_res = pd.DataFrame(index=[], columns=[])
  
  for f in files:
    path = f'/content/drive/MyDrive/INBOX/テスト/{f}'
    df = get_co2(path)
    df_res = pd.concat([df_res, df])

  return df_res

In [ ]:
df_detail = get_dataframe()
df_detail

/usr/local/lib/python3.7/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,種類,使用量,単位,排出係数,CO2排出量,年次,会社名,ID,区分
0,揮発油（ガソリン）,600,ℓ,0.0671,1392,2021年,石塚硝子,専01,燃料
1,軽油,17600,ℓ,0.068567,45495,2021年,石塚硝子,専01,燃料
2,液化石油ガス(ＬＰＧ),660,kg,0.059033,1979,2021年,石塚硝子,専01,燃料
3,都市ガス(13A),14923820,Nm3,0.049867,33489052,2021年,石塚硝子,専01,燃料
4,石灰石使用量,6717.9,ｔ,440,2955876,2021年,石塚硝子,専01,工程
...,...,...,...,...,...,...,...,...,...
30,関西電力（ｾﾞﾛｶｰﾎﾞﾝﾒﾆｭｰを除く）,244837542,kWh,0.475,116297832,2013年,神戸製鋼所加古川製鉄所,専43,電気
0,都市ガス(13A),2380000,Nm3,0.049867,5340720,2021年,日本水産,専19,燃料
1,関西電力（ｾﾞﾛｶｰﾎﾞﾝﾒﾆｭｰを除く）,15306000,kWh,0.351,5372406,2021年,日本水産,専19,電気
2,都市ガス(13A),1834891,Nm3,0.049867,4117495,2013年,日本水産,専19,燃料


合計値を確認

In [ ]:
df_sum = df_detail.copy().groupby(['年次']).agg({'CO2排出量': 'sum'}).reset_index()
df_sum

,年次,CO2排出量
0,2013年,30290839889
1,2021年,25498606118


電気のCO2排出係数を0.250にした場合

## CO2排出明細の作成（電気0.250）

2030年に電気の排出係数が0.250になった場合の想定値

電気使用量は2021年とする

In [ ]:
def co2_2030(df_arg):

  df = df_arg.copy()
  
  # 2021年を抽出
  df = df[df['年次'] == '2021年']

  # 電気の排出係数を0.250に置換
  df['排出係数'] = df.apply(lambda x: x['排出係数'] if x['区分'] != '電気' else 0.250, 1)

  # 年次を2030年に置換
  df = df.replace({'年次': {'2021年': '2030年'}})

  return df

In [ ]:
df_detail_2030 = co2_2030(df_detail)
df_detail_2030

,種類,使用量,単位,排出係数,CO2排出量,年次,会社名,ID,区分
0,揮発油（ガソリン）,600,ℓ,0.067100,1392,2030年,石塚硝子,専01,燃料
1,軽油,17600,ℓ,0.068567,45495,2030年,石塚硝子,専01,燃料
2,液化石油ガス(ＬＰＧ),660,kg,0.059033,1979,2030年,石塚硝子,専01,燃料
3,都市ガス(13A),14923820,Nm3,0.049867,33489052,2030年,石塚硝子,専01,燃料
4,石灰石使用量,6717.9,ｔ,440.000000,2955876,2030年,石塚硝子,専01,工程
...,...,...,...,...,...,...,...,...,...
14,一般電気事業者（売電）,-283424000,kWh,0.250000,-203789052,2030年,神戸製鋼所加古川製鉄所,専43,電気
15,関西電力（ｾﾞﾛｶｰﾎﾞﾝﾒﾆｭｰを除く）,85068000,kWh,0.250000,29858868,2030年,神戸製鋼所加古川製鉄所,専43,電気
16,関西電力（ｾﾞﾛｶｰﾎﾞﾝﾒﾆｭｰを除く）,70695750,kWh,0.250000,24814208,2030年,神戸製鋼所加古川製鉄所,専43,電気
0,都市ガス(13A),2380000,Nm3,0.049867,5340720,2030年,日本水産,専19,燃料


## データ整理

集計して整理

In [ ]:
def sum_co2(time,df_arg):
  
  df = df_arg.copy()
  
  # 単位をtに変換
  df['CO2排出量'] = df.apply(lambda x: x['CO2排出量'] / 1000, 1)
  df['CO2排出量'] = df['CO2排出量'].astype('int')
  
  # データ整形
  df = df.copy().groupby(['ID','会社名','年次','区分']).agg({'CO2排出量': 'sum'}).reset_index()
  df = df[df['年次'] == time]
  df = df.pivot(index=['ID','会社名'], columns='区分', values='CO2排出量').reset_index()
  df['合計'] = df.sum(axis=1)
  
  # 列名の変更
  columns = {'工程':f'{time}_工程【t】','燃料':f'{time}_燃料【t】','電気':f'{time}_電気【t】','合計':f'{time}_合計【t】'}
  df = df.rename(columns=columns)
  
  return df

In [ ]:

# 2013年、2021年、2030年をマージ
df_2013 = sum_co2('2013年',df_detail)
df_2021 = sum_co2('2021年',df_detail)

df = pd.merge(df_2013, df_2021, on=['ID','会社名'] , how='left')

df_2030 = sum_co2('2030年',df_detail_2030)

df = pd.merge(df, df_2030, on=['ID','会社名'] , how='left')

# 算出方法の列を追加
df = df.assign(算出方法 = 'ｱﾝｹｰﾄ')

# 環境政策課データを追加する
df_kankyou = pd.read_csv('/content/drive/MyDrive/INBOX/環境政策課データ.csv', encoding = "shift-jis")
df = pd.concat([df, df_kankyou], axis=0)

# IDでソート
df = df.sort_values('ID')

# 合計行を追加
df = df.append(df.sum(numeric_only=True), ignore_index=True)

df = df.fillna(0)

df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  from ipykernel import kernelapp as app


,ID,会社名,2013年_工程【t】,2013年_燃料【t】,2013年_電気【t】,2013年_合計【t】,2021年_工程【t】,2021年_燃料【t】,2021年_電気【t】,2021年_合計【t】,2030年_工程【t】,2030年_燃料【t】,2030年_電気【t】,2030年_合計【t】,算出方法
0,専01,石塚硝子,7669.0,38273.0,14774.0,60716.0,5859.0,33536.0,11249.0,50644.0,5859.0,33536.0,11249.0,50644.0,ｱﾝｹｰﾄ
1,専02,日本触媒,27564.0,119587.0,79249.0,226400.0,33949.0,157067.0,30607.0,221623.0,33949.0,157067.0,30607.0,221623.0,ｱﾝｹｰﾄ
2,専03,JFE条鋼,2936.0,54060.0,161213.0,218209.0,2778.0,40445.0,87573.0,130796.0,2778.0,40445.0,87573.0,130796.0,ｱﾝｹｰﾄ
3,専04,ケミプロ化成,0,3286.0,1832.0,5118.0,0,2962.0,1568.0,4530.0,0,2962.0,1568.0,4530.0,ｱﾝｹｰﾄ
4,専06,ダイセル,0,661629.0,-3072.0,658557.0,0,608147.0,-10263.0,597884.0,0,608147.0,-10263.0,597884.0,ｱﾝｹｰﾄ
5,専07,関西電力,0,10089166.0,0,10089166.0,6986760.0,62.0,0,6986822.0,6986760.0,62.0,0,6986822.0,ｱﾝｹｰﾄ
6,専09,合同製鐵,"7,385","28,095","91,948","127,428","9,040","29,563","68,696","107,299","7,385","28,095","48,394","83,874",環境政策課
7,専10,兵庫県揖保川浄化センター,0,7.0,13236.0,13243.0,0,4.0,6282.0,6286.0,0,4.0,6282.0,6286.0,ｱﾝｹｰﾄ
8,専11,兵庫県兵庫西スラッジセンター,0,10967.0,14991.0,25958.0,0,6600.0,9556.0,16156.0,0,6600.0,9556.0,16156.0,ｱﾝｹｰﾄ
9,専12,山陽特殊製鋼,0,342529.0,473904.0,816433.0,0,345284.0,364200.0,709484.0,0,345284.0,364200.0,709484.0,ｱﾝｹｰﾄ


# 専用埠頭の荷役機械等

## 単体チェック

EXCELをインポートして整形する関数

In [ ]:
def get_co2(path):

  # Bookをインポートしてシート名をリストに格納
  book = pd.ExcelFile(path)
  sheets = book.sheet_names

  # ファイル名からIDと事業者名を取得
  filename = path.split('/')[7]
  id = filename.split('_')[0]
  company = filename.split('_')[1].replace('.xlsx', '')
  
  # CO2排出量のシートをDataFrameに格納
  df = book.parse(sheets[4])

  # 必要な行だけ抽出
  df = df.dropna(subset=['Unnamed: 16']).reset_index(drop=True)
  
  # 列の整理
  df = df[['アンケートⅡ　CO2排出量と削減計画等','Unnamed: 4','Unnamed: 9','Unnamed: 15','Unnamed: 16','Unnamed: 17','Unnamed: 19']]
  df = df.rename(columns={'アンケートⅡ　CO2排出量と削減計画等' : '名称','Unnamed: 4' : '種類','Unnamed: 9' : '使用量','Unnamed: 15' : '単位','Unnamed: 16' : '荷役運転時間','Unnamed: 17' : '取扱貨物種類','Unnamed: 19' : '取扱貨物量'})
  
  # # 余計なデータを削除
  # df = df[df['CO2排出量'] != '－']

  df = df.assign(年次 = '',会社名 = company,ID = id)
  
  for index, row in df.iterrows():
    
    if type(row['荷役運転時間']) is str:
      time = '2021年' if index == 0 else '2013年'
      df = df.drop(index)
  
    df['年次'][index] = time

  # # 荷役機械のデータが紛れ込むので削除
  # df = df[df['排出係数'] != 'LPG']
  # df = df[df['排出係数'] != 'ケミカル']
  # df = df[df['排出係数'] != '石炭']
  # df = df[df['排出係数'] != 'バン頁']
  # df = df[df['排出係数'] != '珪砂']
  
  # # 改行を削除
  # df = df.replace( '\n', '', regex=True)

  # # CO2排出量を整数に変換
  # df['CO2排出量'] = df['CO2排出量'].astype('int')

  # # 排出区分を追加する
  # df_type = pd.read_csv('/content/drive/MyDrive/INBOX/企業活動排出区分.csv', encoding = "shift-jis")
  # df_type = df_type[~df_type.duplicated()]
 
  # df = pd.merge(df, df_type, on=['種類'] , how='left')
  # df = df.fillna({'区分': '電気'})

  return df


パスを指定してテスト

In [ ]:
path = '/content/drive/MyDrive/播磨臨海地域CNP/CO2排出量の推計/企業アンケート結果/専05_ダイセル.xlsx'
df = get_co2(path)
df

/usr/local/lib/python3.7/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,アンケートⅡ CO2排出量と削減計画等,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,１,現在(2021年度),NaN,のCO2排出量について,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,（１）事業所で管理している施設からのCO2排出量について、ご記入ください。\n 「環境の...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,事業所名\n※自動入力,NaN,NaN,NaN,燃料・焼却物等の種類名称\n（別表②参照）,NaN,NaN,NaN,NaN,使用量\n（年当たり）,...,NaN,NaN,単位\n(㍑、kwh、kg、MJ),NaN,排出係数,NaN,CO2換算\n排出量\n(kg-CO2) ※自動計算,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
92,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 荷役機械明細の作成

In [ ]:
def get_dataframe():

  files = os.listdir('drive/MyDrive/播磨臨海地域CNP/CO2排出量の推計/企業アンケート結果/')
  
  df_res = pd.DataFrame(index=[], columns=[])
  
  for f in files:
    path = f'/content/drive/MyDrive/播磨臨海地域CNP/CO2排出量の推計/企業アンケート結果/{f}'
    df = get_co2(path)
    df_res = pd.concat([df_res, df])

  return df_res

In [ ]:
df_detail = get_dataframe()
df_detail

/usr/local/lib/python3.7/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,名称,種類,使用量,単位,荷役運転時間,取扱貨物種類,取扱貨物量,年次,会社名,ID
1,5t水平引き込みクレーン,電気,14092.5,NaN,140.925,バン頁,6769,2021年,AGC,専52
3,5t水平引き込みクレーン,電気,11476.25,NaN,114.7625,バン頁,2961,2013年,AGC,専52
1,船側ポンプ,Ａ重油,212440,ℓ,1880,LPG,389715,2021年,日本触媒,専02
2,船側ポンプ,Ａ重油,159152,ℓ,2030,ケミカル,333115,2021年,日本触媒,専02
3,陸上ポンプ,関西電力\n（ｾﾞﾛｶｰﾎﾞﾝﾒﾆｭｰを除く）,43420,kWh,835,ケミカル,203901,2021年,日本触媒,専02
1,アンローダー\n230t/h,NaN,電源は自家発電電気で充当,NaN,1000,石炭,211530,2021年,カネカ,専48
2,"クレーン180t/h,230t/h",NaN,電源は自家発電電気で充当,NaN,2364,原塩,510829,2021年,カネカ,専48
4,アンローダー\n230t/h,NaN,電源は自家発電電気で充当,NaN,1247,石炭,254066,2013年,カネカ,専48
5,"クレーン\n180t/h,230t/h",NaN,電源は自家発電電気で充当,NaN,1816,原塩,396670,2013年,カネカ,専48
1,揚炭機,関西電力（ｾﾞﾛｶｰﾎﾞﾝﾒﾆｭｰ)以外\nの電気事業者,－,kWh,1560,石炭,1167000,2021年,J-POWER,専51


# 専用岸壁の船舶

## 単体チェック

EXCELをインポートして整形する関数

In [53]:
def get_ships(path):

  # Bookをインポートしてシート名をリストに格納
  book = pd.ExcelFile(path)
  sheets = book.sheet_names

  # ファイル名からIDと事業者名を取得
  filename = path.split('/')[7]
  id = filename.split('_')[0]
  company = filename.split('_')[1].replace('.xlsx', '')
  
  # CO2排出量のシートをDataFrameに格納
  df = book.parse(sheets[4])

  # 必要な行だけ抽出
  df = df.dropna(subset=['Unnamed: 6']).reset_index(drop=True)
  
  # # 列の整理
  # df = df[['アンケートⅡ　CO2排出量と削減計画等','Unnamed: 4','Unnamed: 9','Unnamed: 15','Unnamed: 16','Unnamed: 17','Unnamed: 19']]
  # df = df.rename(columns={'アンケートⅡ　CO2排出量と削減計画等' : '名称','Unnamed: 4' : '種類','Unnamed: 9' : '使用量','Unnamed: 15' : '単位','Unnamed: 16' : '荷役運転時間','Unnamed: 17' : '取扱貨物種類','Unnamed: 19' : '取扱貨物量'})
  
  # # # 余計なデータを削除
  # # df = df[df['CO2排出量'] != '－']

  # df = df.assign(年次 = '',会社名 = company,ID = id)
  
  # for index, row in df.iterrows():
    
  #   if type(row['荷役運転時間']) is str:
  #     time = '2021年' if index == 0 else '2013年'
  #     df = df.drop(index)
  
  #   df['年次'][index] = time

  # # 荷役機械のデータが紛れ込むので削除
  # df = df[df['排出係数'] != 'LPG']
  # df = df[df['排出係数'] != 'ケミカル']
  # df = df[df['排出係数'] != '石炭']
  # df = df[df['排出係数'] != 'バン頁']
  # df = df[df['排出係数'] != '珪砂']
  
  # # 改行を削除
  # df = df.replace( '\n', '', regex=True)

  # # CO2排出量を整数に変換
  # df['CO2排出量'] = df['CO2排出量'].astype('int')

  # # 排出区分を追加する
  # df_type = pd.read_csv('/content/drive/MyDrive/INBOX/企業活動排出区分.csv', encoding = "shift-jis")
  # df_type = df_type[~df_type.duplicated()]
 
  # df = pd.merge(df, df_type, on=['種類'] , how='left')
  # df = df.fillna({'区分': '電気'})

  return df


パスを指定してテスト

In [54]:
path = '/content/drive/MyDrive/播磨臨海地域CNP/CO2排出量の推計/企業アンケート結果/専02_日本触媒.xlsx'
df = get_ships(path)
df

/usr/local/lib/python3.7/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,アンケートⅡ CO2排出量と削減計画等,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29
0,船種,NaN,NaN,総トン数(階級別),NaN,NaN,総停泊時間,NaN,CO2排出量(kg-CO2),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ＬＰＧ船(内航船),NaN,NaN,773.2,NaN,NaN,1751.6,NaN,516847.782186,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ＬＰＧ船(外航船),NaN,NaN,1273.3,NaN,NaN,77.4,NaN,16422.408073,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,内航ケミカル船(内航船),NaN,NaN,489.2,NaN,NaN,2082,NaN,428951.031062,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,外航ケミカル船(外航船),NaN,NaN,773.4,NaN,NaN,85.2,NaN,12482.868205,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,２,2014年度,のCO2排出量について,NaN,NaN,NaN,※現況(2021年度)と変わらないのであれば空欄で結構です。,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,※2013年度当時に立地されていなかった企業様は空欄で結構です。,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,船種,NaN,NaN,総トン数(階級別),NaN,NaN,総停泊時間,NaN,CO2排出量(kg-CO2),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,ＬＰＧ船(内航船),NaN,NaN,753.3,NaN,NaN,663.4,NaN,191743.812845,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,ＬＰＧ船(外航船),NaN,NaN,798,NaN,NaN,54,NaN,8094.689958,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
